### 导入数据分析和可视化所需的库

In [ ]:
# 导入数据分析和可视化所需的库
import numpy as np  # 用于数值计算
import matplotlib.pyplot as plt  # 用于绘图
import pandas as pd  # 用于数据处理
import seaborn as sns  # 用于统计数据可视化
import lightgbm as lgb  # 用于LightGBM模型
from scipy import stats  # 用于统计分析
from sklearn.model_selection import train_test_split  # 用于数据集划分
from sklearn.metrics import mean_squared_error  # 用于均方误差计算
from sklearn.linear_model import Ridge  # 用于岭回归
from sklearn.linear_model import Lasso  # 用于Lasso回归
from sklearn.svm import SVR  # 用于支持向量机回归
from xgboost import XGBRegressor  # 用于XGBoost回归
from statsmodels.stats.outliers_influence import variance_inflation_factor  # 用于计算方差膨胀因子
from sklearn.decomposition import PCA  # 用于主成分分析
from sklearn.linear_model import LinearRegression  # 用于线性回归模型
from sklearn.neighbors import KNeighborsRegressor  # 用于K近邻回归
from sklearn.tree import DecisionTreeRegressor  # 用于决策树回归
from sklearn.ensemble import RandomForestRegressor  # 用于随机森林回归
from sklearn.ensemble import GradientBoostingRegressor  # 用于梯度提升回归
import warnings

warnings.filterwarnings('ignore')  # 忽略警告信息

### 读取数据

In [ ]:
# 定义数据文件路径
train_data_file = '/Users/zhuzijie/Downloads/zhengqi/zhengqi_train.txt'
test_data_file = '/Users/zhuzijie/Downloads/zhengqi/zhengqi_test.txt'
# 读取训练和测试数据集（以制表符分隔）
train_data = pd.read_csv(train_data_file, sep='\t')
test_data = pd.read_csv(test_data_file, sep='\t')
# 显示训练数据的统计摘要
train_data.describe()

### 数据预处理

##### 绘制箱线图

In [ ]:
# 为V0特征创建单独的箱线图
fig_test = plt.figure(figsize=(12, 8))  # 设置图形大小
sns.boxplot(x=train_data['V0'], width=0.5)  # 绘制箱线图，宽度为0.5
plt.savefig('./2-特征箱式图.jpg', dpi=300)  # 保存图形为高分辨率图片

In [ ]:
# 为所有特征绘制箱线图
# 获取所有特征（不包括目标变量）
features = train_data.columns.tolist()[:38]  # 确保只包含38个特征
num_features = len(features)

# 计算布局
n_cols = 3
n_rows = (num_features + n_cols - 1) // n_cols  # 向上取整

# 创建子图布局
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 2.5))
axes = axes.flatten()  # 将二维数组展平为一维，便于索引

# 设置全局标题
fig.suptitle('各特征的分布情况（箱线图）', fontsize=16, y=0.92)

# 设置字体
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

# 统一样式参数
box_props = {
    'boxprops': {'facecolor': '#1f77b4', 'edgecolor': 'black', 'alpha': 0.7},
    'medianprops': {'color': 'red'},
    'whiskerprops': {'color': 'black'},
    'capprops': {'color': 'black'}
}

# 遍历每个特征绘制箱线图
for i, feature in enumerate(features):
    # 在当前子图上绘制箱线图
    sns.boxplot(x=train_data[feature], ax=axes[i], width=0.6, **box_props)

    # 添加网格线
    axes[i].grid(True, linestyle='--', alpha=0.3)

    # 设置标题和轴标签
    axes[i].set_title(feature, fontsize=10)
    axes[i].set_xlabel('值', fontsize=8)
    axes[i].set_ylabel('频率', fontsize=8)
    axes[i].tick_params(axis='both', labelsize=7)

# 隐藏多余的子图
for i in range(num_features, len(axes)):
    axes[i].set_visible(False)

# 调整布局
plt.tight_layout()
plt.subplots_adjust(top=0.9)  # 为标题留出空间

# 保存高分辨率图片
plt.savefig('./特征箱线图.png', dpi=300, bbox_inches='tight')

plt.show()

plt.close(fig)

##### 绘制特征分布对比图

In [ ]:
# 创建单特征分布对比图：训练集与测试集的V0特征
# 创建图形和坐标轴
fig, ax = plt.subplots(figsize=(10, 6))

# 用来正常显示中文标签
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

# 定义一致的颜色方案
train_color = '#E41A1C'  # 红色
test_color = '#377EB8'  # 蓝色

# 绘制核密度估计图
sns.kdeplot(train_data['V0'], color=train_color, fill=True, alpha=0.8, ax=ax, label='训练集')
sns.kdeplot(test_data['V0'], color=test_color, fill=True, alpha=0.8, ax=ax, label='测试集')

# 设置图表标题和标签
ax.set_title('V0特征的分布对比', fontsize=14, pad=10)
ax.set_xlabel('V0值', fontsize=12)
ax.set_ylabel('密度', fontsize=12)

# 添加图例
ax.legend(fontsize=10)

# 优化刻度标签大小
ax.tick_params(axis='both', labelsize=10)

# 添加网格线提高可读性
ax.grid(True, linestyle='--', alpha=0.3)

# 调整布局
plt.tight_layout()

# 保存高分辨率图片
plt.savefig('./V0特征分布对比.jpg', dpi=300, bbox_inches='tight')

# 显示图形
plt.show()

plt.close(fig)

In [ ]:
# 创建多个子图比较训练集和测试集的特征分布
# 计算布局参数
dist_cols = 6
dist_rows = (len(test_data.columns) + dist_cols - 1) // dist_cols  # 向上取整确保空间足够

# 创建图形和轴对象数组
fig, axes = plt.subplots(dist_rows, dist_cols, figsize=(4 * dist_cols, 3.5 * dist_rows))
axes = axes.flatten()  # 将二维数组展平为一维，便于索引

# 用来正常显示中文标签
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

# 设置全局标题
fig.suptitle('特征分布对比: 训练集 vs 测试集', fontsize=16, y=0.92)

# 定义一致的颜色和样式
train_color = '#E41A1C'  # 红色
test_color = '#377EB8'  # 蓝色

# 遍历所有特征
for i, col in enumerate(test_data.columns):
    # 在当前子图上绘制核密度估计
    sns.kdeplot(train_data[col], color=train_color, fill=True, alpha=0.8, ax=axes[i], label='训练集')
    sns.kdeplot(test_data[col], color=test_color, fill=True, alpha=0.8, ax=axes[i], label='测试集')

    # 设置轴标签
    axes[i].set_xlabel(col, fontsize=10)
    axes[i].set_ylabel('密度', fontsize=10)
    axes[i].legend(fontsize=8)
    axes[i].tick_params(axis='both', labelsize=8)  # 调整刻度标签大小

# 隐藏多余的子图
for i in range(len(test_data.columns), len(axes)):
    axes[i].set_visible(False)

# 调整子图之间的间距
plt.tight_layout()
plt.subplots_adjust(top=0.9)  # 为标题留出空间

# 保存高分辨率图片
plt.savefig('./特征分布对比.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close(fig)

##### 特征相关性分析

In [ ]:
# 特征变量相关性
drop_col_kde = ['V5', 'V9', 'V11', 'V17', 'V22', 'V28']
train_data_drop = train_data.drop(columns=drop_col_kde)
train_corr = train_data_drop.corr()
train_corr

In [ ]:
# 绘制热力图显示特征之间的相关性
fig, ax = plt.subplots(figsize=(20, 16))  # 创建一个大尺寸的图形(20x20英寸)以容纳所有相关性数据
sns.heatmap(train_corr, vmax=.8, square=True, annot=True, ax=ax)  # 绘制相关性热力图，设置最大颜色值为0.8，使用方形单元格，并在每个单元格中显示具体的相关系数值

In [ ]:
# 创建优化的相关性热力图（下三角矩阵）
plt.figure(figsize=(20, 16), clear=True)

# 计算相关性矩阵
corr_matrix = train_data_drop.corr()

# 创建掩码以仅显示下三角矩阵，避免信息重复
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# 创建更美观的颜色映射
cmap = sns.diverging_palette(230, 20, as_cmap=True)  # 蓝-白-红配色方案

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

# 绘制热力图
sns.heatmap(
    corr_matrix,
    mask=mask,
    cmap=cmap,
    vmin=-1,  # 设置颜色范围
    vmax=1,
    center=0,  # 将白色设为0值
    square=True,  # 确保单元格为正方形
    annot=True,  # 显示相关系数
    fmt='.2f',  # 保留两位小数
    linewidths=0.5,  # 设置网格线宽度
    cbar_kws={'shrink': 0.8, 'label': '相关系数'}  # 自定义颜色条
)

# 设置标题
plt.title('特征相关性分析热力图', fontsize=16, pad=10)

# 优化布局
plt.tight_layout()

# 保存高分辨率图片
plt.savefig('./特征相关性分析.png', dpi=300, bbox_inches='tight')

# 显示图形
plt.show()

##### 特征删除

In [ ]:
# 根据数据分布进行特征删除
train_data.drop(drop_col_kde, axis=1, inplace=True)
test_data.drop(drop_col_kde, axis=1, inplace=True)
train_data.head()

In [ ]:
# 根据相关性系数进行特征筛选
cond = corr_matrix['target'].abs() < 0.1
drop_col_corr = corr_matrix.index[cond]
train_data.drop(drop_col_corr, axis=1, inplace=True)
test_data.drop(drop_col_corr, axis=1, inplace=True)
train_data.head()

##### 数据整合

In [ ]:
# 整合训练数据和测试数据
train_data['label'] = 'train'
test_data['label'] = 'test'
all_data = pd.concat([train_data, test_data])
all_data.to_csv('./processed_zhengqi_data.csv', index=False)

##### 特征归一化

In [ ]:
# 特征归一化
columns = list(all_data.columns)
# 删除标签列索引
columns.remove('label')
# 删除目标值
columns.remove('target')
all_data[columns].head()

In [ ]:
# 定义归一化
def norm_min_max(data):
    return (data - data.min()) / (data.max() - data.min())


all_data[columns] = all_data[columns].apply(norm_min_max, axis=0)
all_data.describe()

##### 特征分布分析

In [ ]:
# 获取训练数据
cond = all_data['label'] == 'train'
train_data = all_data[cond]
train_data.drop(labels='label', axis=1, inplace=True)

In [ ]:
# 创建一个宽20英寸、高8英寸的图形画布
plt.figure(figsize=(20, 8))

# 第一部分：绘制特征V0的分布直方图
ax1 = plt.subplot(1, 3, 1)  # 创建1行3列的子图布局，选择第1个位置
sns.histplot(data=train_data, x='V0', kde=True, stat='density', ax=ax1)  # 绘制V0的直方图，并叠加核密度估计曲线
# 添加正态分布曲线
x = np.linspace(train_data['V0'].min(), train_data['V0'].max(), 100)  # 创建从V0最小值到最大值的100个等间距点
y = stats.norm.pdf(x, train_data['V0'].mean(), train_data['V0'].std())  # 计算这些点对应的正态分布概率密度值
# 绘制正态分布拟合曲线，并进行缩放以匹配直方图高度
ax1.plot(x, y, color='red', lw=2)
ax1.set_title('V0分布图')  # 设置子图标题

# 第二部分：绘制Q-Q图(Quantile-Quantile plot)检验数据是否符合正态分布
ax2 = plt.subplot(1, 3, 2)  # 选择第2个子图位置
valid_data = train_data['V0'].dropna()  # 去除V0中可能的缺失值
stats.probplot(valid_data, plot=ax2)  # 绘制Q-Q图，比较数据分位数与理论正态分布分位数
ax2.set_title(f'skew={stats.skew(valid_data):.4f}')  # 显示偏度值，判断分布偏离正态的程度

# 第三部分：绘制特征V0与目标变量target的散点图，分析相关性
ax3 = plt.subplot(1, 3, 3)  # 选择第3个子图位置
ax3.scatter(train_data['V0'], train_data['target'], s=5, alpha=0.5)  # 绘制散点图，点大小为5，透明度为0.5
corr = np.corrcoef(train_data['V0'].values, train_data['target'].values)[0, 1]  # 计算皮尔逊相关系数
ax3.set_title(f'corr={corr:.4f}')  # 在子图标题中显示相关系数，保留4位小数

plt.tight_layout()  # 自动调整子图参数，使之填充整个图像区域且子图之间不重叠
plt.savefig('./6-Box-Cox.jpg', dpi=300, bbox_inches='tight')  # 保存高分辨率图像，确保边界紧凑
plt.show()  # 显示图形

In [ ]:
cols = 3  # 每行的子图数量
columns = list(train_data.columns)
columns.remove('target')
rows = len(columns)  # 特征数量
plt.figure(figsize=(5 * cols, 4 * rows))  # 创建一个宽15英寸、高4乘以特征数量英寸的图形画布

for i, feature in enumerate(columns):
    row_position = i * cols + 1  # 计算当前特征在图中的起始位置

    # 第一部分：绘制特征的分布直方图
    ax1 = plt.subplot(rows, cols, row_position)
    sns.histplot(data=train_data, x=feature, kde=True, stat='density', ax=ax1)
    # 添加正态分布曲线
    x = np.linspace(train_data[feature].min(), train_data[feature].max(), 100)
    y = stats.norm.pdf(x, train_data[feature].mean(), train_data[feature].std())
    # 绘制正态分布曲线，无需缩放因为使用了stat='density'
    ax1.plot(x, y, color='red', lw=2)
    ax1.set_title(f'{feature}分布图')
    ax1.set_xlabel('')
    ax1.set_ylabel('')

    # 第二部分：绘制Q-Q图
    ax2 = plt.subplot(rows, cols, row_position + 1)
    valid_data = train_data[feature].dropna()
    stats.probplot(valid_data, plot=ax2)
    ax2.set_title(f'skew={stats.skew(valid_data):.4f}')
    ax2.set_xlabel('')
    ax2.set_ylabel('')

    # 第三部分：绘制特征与目标变量target的散点图
    ax3 = plt.subplot(rows, cols, row_position + 2)
    ax3.scatter(train_data[feature], train_data['target'], s=5, alpha=0.5)
    corr = np.corrcoef(train_data[feature].values, train_data['target'].values)[0, 1]
    ax3.set_title(f'corr={corr:.2f}')

plt.tight_layout()
plt.savefig('./features_analysis.jpg', dpi=300, bbox_inches='tight')
plt.show()

##### Box-Cox变换

In [ ]:
# Box-Cox变换
cols = 6  # 每行的子图数量
columns = list(train_data.columns)
columns.remove('target')
rows = len(columns)  # 特征数量
plt.figure(figsize=(5 * cols, 4 * rows))


def norm_min_max(data):
    return (data - data.min()) / (data.max() - data.min())


for i, feature in enumerate(columns):
    row_position = i * cols + 1

    # 第一部分：绘制特征的原始分布直方图
    ax1 = plt.subplot(rows, cols, row_position)
    sns.histplot(data=train_data, x=feature, kde=True, stat='density', ax=ax1)
    x = np.linspace(train_data[feature].min(), train_data[feature].max(), 100)
    y = stats.norm.pdf(x, train_data[feature].mean(), train_data[feature].std())
    ax1.plot(x, y, color='red', lw=2)
    ax1.set_title(f'{feature}分布图-raw')
    ax1.set_xlabel('')
    ax1.set_ylabel('')

    # 第二部分：绘制原始特征Q-Q图
    ax2 = plt.subplot(rows, cols, row_position + 1)
    valid_data = train_data[feature].dropna()
    stats.probplot(valid_data, plot=ax2)
    ax2.set_title(f'skew={stats.skew(valid_data):.4f}-raw')
    ax2.set_xlabel('')
    ax2.set_ylabel('')

    # 第三部分：绘制原始特征与目标变量的散点图
    ax3 = plt.subplot(rows, cols, row_position + 2)
    ax3.scatter(train_data[feature], train_data['target'], s=5, alpha=0.5)
    corr = np.corrcoef(train_data[feature].values, train_data['target'].values)[0, 1]
    ax3.set_title(f'corr={corr:.2f}-raw')

    # Box-Cox变换（确保数据为正值）
    feature_data = train_data[feature].dropna()
    if feature_data.min() <= 0:
        feature_data = feature_data - feature_data.min() + 1  # 确保全部为正值

    # 应用Box-Cox变换
    transformed_data, lambda_val = stats.boxcox(feature_data)

    # 归一化变换后的数据
    transformed_norm = norm_min_max(transformed_data)

    # 第四部分：变换后的分布直方图
    ax4 = plt.subplot(rows, cols, row_position + 3)
    sns.histplot(transformed_norm, kde=True, stat='density', ax=ax4)
    x_trans = np.linspace(min(transformed_norm), max(transformed_norm), 100)
    y_trans = stats.norm.pdf(x_trans, np.mean(transformed_norm), np.std(transformed_norm))
    ax4.plot(x_trans, y_trans, color='red', lw=2)
    ax4.set_title(f'{feature}分布图-Box-Cox')
    ax4.set_xlabel('')
    ax4.set_ylabel('')

    # 第五部分：变换后的Q-Q图
    ax5 = plt.subplot(rows, cols, row_position + 4)
    stats.probplot(transformed_norm, plot=ax5)
    ax5.set_title(f'skew={stats.skew(transformed_norm):.4f}-Box-Cox')
    ax5.set_xlabel('')
    ax5.set_ylabel('')

    # 第六部分：变换后的特征与目标变量的散点图
    ax6 = plt.subplot(rows, cols, row_position + 5)
    # 获取对应的目标变量值
    target_values = train_data.loc[feature_data.index, 'target'].values
    ax6.scatter(transformed_norm, target_values, s=5, alpha=0.5)
    corr_trans = np.corrcoef(transformed_norm, target_values)[0, 1]
    ax6.set_title(f'corr={corr_trans:.2f}-Box-Cox')

plt.tight_layout()
plt.savefig('./features_Box_Cox_analysis.jpg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
columns = list(all_data.columns)
# 删除标签列索引
columns.remove('label')
# 删除目标值
columns.remove('target')

for col in columns:
    all_data[col], lambda_val = stats.boxcox(all_data[col] + 1)

### 模型训练

##### 数据集划分

In [ ]:
def get_train_data(data=None):
    """
    获取训练数据集的特征和目标变量

    参数:
    data - 数据集DataFrame，如果为None则使用全局的all_data变量

    返回:
    X - 特征矩阵
    y - 目标变量
    """
    # 如果未提供数据，则使用全局变量
    if data is None:
        data = all_data

    # 筛选出训练数据
    train_data = data[data['label'] == 'train']
    # 提取目标变量
    y = train_data['target']
    # 提取特征变量（排除目标变量和标签列）
    X = train_data.drop(['target', 'label'], axis=1)

    return X, y


def split_train_test(test_size=0.2, random_state=42, data=None):
    """
    将训练数据集拆分为训练集和验证集

    参数:
    test_size - 验证集比例，默认为0.2（20%）
    random_state - 随机数种子，用于确保结果可重复，默认为42
    data - 数据集DataFrame，如果为None则使用全局的all_data变量

    返回:
    X_train - 训练集特征
    X_valid - 验证集特征
    y_train - 训练集目标变量
    y_valid - 验证集目标变量
    """
    # 获取训练数据
    X, y = get_train_data(data)

    # 使用sklearn的train_test_split函数进行数据集拆分
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_state)

    return X_train, X_valid, y_train, y_valid


def get_test_data(data=None):
    """
    获取测试数据集的特征

    参数:
    data - 数据集DataFrame，如果为None则使用全局的all_data变量

    返回:
    X_test - 测试集特征
    """
    # 如果未提供数据，则使用全局变量
    if data is None:
        data = all_data

    # 筛选出测试数据并重置索引
    test_data = data[data['label'] == 'test'].reset_index(drop=True)
    # 提取特征变量（排除标签列，测试集可能没有目标列）
    drop_cols = ['label']
    if 'target' in test_data.columns:
        drop_cols.append('target')
    X_test = test_data.drop(drop_cols, axis=1)

    return X_test

##### 异常值检测

In [ ]:
def find_outliers(model, X, y, sigma=3):
    """
    使用回归模型检测数据集中的异常值

    参数:
    model - 回归模型对象（如Ridge, Lasso等）
    X - 特征数据
    y - 目标变量
    sigma - 标准差阈值，超过此值被视为异常值

    返回:
    outliers - 异常值的索引
    """
    # 模型训练
    model.fit(X, y)
    y_pred = pd.Series(model.predict(X), index=y.index)  # 预测值，保持与原始数据相同的索引

    # 计算残差（真实值与预测值之差）
    resid = y - y_pred
    mean_resid = resid.mean()  # 残差均值
    std_resid = resid.std()  # 残差标准差

    # 异常值计算：将残差标准化为Z分数
    z = (resid - mean_resid) / std_resid
    outliers = z[abs(z) > sigma].index  # Z分数超过sigma倍标准差的视为异常值

    # 输出模型评价指标
    print('R2=', model.score(X, y))  # 决定系数
    print("mse=", mean_squared_error(y, y_pred))  # 均方误差
    print("------------------------------------")

    # 输出残差统计信息
    print('残差均值:', mean_resid)
    print('残差标准差:', std_resid)
    print("------------------------------------")

    # 输出异常值信息
    print(f'发现{len(outliers)}个异常值:')
    print(outliers.to_list())

    # 创建可视化图表
    plt.figure(figsize=(24, 8))

    # 子图1：真实值与预测值的散点图
    ax131 = plt.subplot(1, 3, 1)
    ax131.scatter(y, y_pred, marker='.', alpha=0.7)  # 绘制所有样本点
    ax131.scatter(y.loc[outliers], y_pred[outliers], color='red', marker='o')  # 突出显示异常值
    ax131.legend(['正常样本', '异常值'])
    ax131.set_xlabel('真实值')
    ax131.set_ylabel('预测值')
    ax131.grid(True, linestyle='--', alpha=0.3)  # 添加网格线

    # 子图2：真实值与残差的散点图
    ax132 = plt.subplot(1, 3, 2)
    ax132.scatter(y, y - y_pred, marker='.', alpha=0.7)  # 绘制所有样本点
    ax132.scatter(y.loc[outliers], y.loc[outliers] - y_pred[outliers], color='red', marker='o')  # 突出显示异常值
    ax132.legend(['正常样本', '异常值'])
    ax132.set_xlabel('真实值')
    ax132.set_ylabel('残差')
    ax132.grid(True, linestyle='--', alpha=0.3)  # 添加网格线
    ax132.axhline(y=0, color='k', linestyle='-', alpha=0.3)  # 添加y=0的参考线

    # 子图3：残差Z分数直方图
    ax133 = plt.subplot(1, 3, 3)
    # 绘制正常样本的Z分数直方图
    ax133.hist(z[~z.index.isin(outliers)], bins=50, alpha=0.7)
    # 绘制异常值的Z分数直方图
    ax133.hist(z.loc[outliers], bins=50, color='red', alpha=0.7)
    ax133.legend(['正常样本', '异常值'])
    ax133.set_xlabel('Z分数')
    ax133.set_ylabel('频数')
    ax133.grid(True, linestyle='--', alpha=0.3)  # 添加网格线

    # 添加异常值阈值参考线
    ax133.axvline(x=sigma, color='r', linestyle='--', alpha=0.5)
    ax133.axvline(x=-sigma, color='r', linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.savefig('./outliers_hist.jpg', dpi=300, bbox_inches='tight')
    return outliers

In [ ]:
# 岭回归算法查找异常值
# 获取训练数据
X_train, y_train = get_train_data()
# 使用岭回归查找异常值
outliers1 = find_outliers(Ridge(), X_train, y_train)

In [ ]:
# 套索回归算法查找异常值
# 获取训练数据
X_train, y_train = get_train_data()
# 使用Lasso查找异常值
outliers2 = find_outliers(Lasso(), X_train, y_train)

In [ ]:
# 支持向量机SVR算法查找异常值
# 获取训练数据
X_train, y_train = get_train_data()
# 使用SVR查找异常值
outliers3 = find_outliers(SVR(), X_train, y_train)

In [ ]:
# Xgboost算法查找异常值
# 获取训练数据
X_train, y_train = get_train_data()
# 使用Xgboost查找异常值
outliers4 = find_outliers(XGBRegressor(), X_train, y_train)

In [ ]:
# 联合所有检测到的异常值
all_outliers = list(set(list(outliers1) + list(outliers2) + list(outliers3) + list(outliers4)))
print(f"Ridge回归检测到 {len(outliers1)} 个异常值")
print(f"Lasso回归检测到 {len(outliers2)} 个异常值")
print(f"SVR检测到 {len(outliers3)} 个异常值")
print(f"XGBoost检测到 {len(outliers4)} 个异常值")
print(f"合并后共有 {len(all_outliers)} 个唯一异常值")

# 创建一个布尔掩码：保留测试数据以及不在异常值列表中的训练数据
keep_mask = ~((all_data['label'] == 'train') & all_data.index.isin(all_outliers))

# 应用掩码筛选数据
all_data_clean = all_data[keep_mask]

# 保存处理后的数据
all_data_clean.to_csv('./processed_zhengqi_data.csv', index=False)
print(f"原始数据形状: {all_data.shape}")
print(f"清洗后数据形状: {all_data_clean.shape}")
print(f"已删除 {all_data.shape[0] - all_data_clean.shape[0]} 行异常数据")

##### VIF分析

In [ ]:
# VIF计算代码
def calculate_vif(df):
    """
    计算数据框中所有特征的方差膨胀因子(VIF)

    参数:
    df - 包含特征变量的DataFrame

    返回:
    vif_df - 包含每个特征VIF值的DataFrame
    """
    # 创建包含特征名和VIF值的DataFrame
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns

    # 计算每个特征的VIF值
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]

    # 排序显示结果
    return vif_data.sort_values("VIF", ascending=False).reset_index(drop=True)


# 获取训练数据
X_train, y_train = get_train_data(all_data_clean)

# 计算并显示VIF值
vif_results = calculate_vif(X_train)
print("方差膨胀因子(VIF)分析结果:")
display(vif_results)

##### PCA降维

In [ ]:
# PCA降维
def perform_pca_analysis(X, n_components=22):
    """
    执行PCA降维分析并计算降维后的VIF值

    参数:
    X - 输入特征矩阵
    n_components - PCA组件数量
    plot_variance - 是否绘制解释方差图，默认True

    返回:
    X_pca - 降维后的特征矩阵
    pca_model - 训练好的PCA模型
    """
    # 1. 创建PCA模型
    # whiten=True: 白化处理，使各主成分具有相同的方差
    pca = PCA(n_components=n_components, whiten=True)

    # 2. 拟合模型并转换数据
    X_pca = pca.fit_transform(X)

    # 3. 分析并展示PCA结果
    explained_variance = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance)

    print(f"使用{n_components}个主成分，共解释了{cumulative_variance[-1] * 100:.2f}%的总方差")

    return X_pca, pca


# 计算降维后的VIF值
def calculate_vif_after_pca(X_pca):
    """
    计算PCA降维后各主成分的方差膨胀因子(VIF)

    参数:
    X_pca - PCA降维后的数据

    返回:
    vif_df - 包含VIF值的DataFrame
    """
    # 创建包含主成分编号和VIF值的DataFrame
    vif_data = pd.DataFrame()
    vif_data["主成分"] = [f"PC{i + 1}" for i in range(X_pca.shape[1])]

    # 计算每个主成分的VIF值
    vif_values = [variance_inflation_factor(X_pca, i) for i in range(X_pca.shape[1])]
    vif_data["VIF"] = [round(val, 2) for val in vif_values]  # 四舍五入到两位小数

    return vif_data


# 获取训练数据
X_train, y_train = get_train_data(all_data_clean)
# 执行PCA分析
X_train_pca, pca_model = perform_pca_analysis(X_train, n_components=22)
# 计算并显示降维后的VIF值
vif_results = calculate_vif_after_pca(X_train_pca)
print("PCA降维后主成分的VIF值分析:")
display(vif_results)

# 保存降维后的数据
np.savez('./train_data_pca', X_train=X_train_pca, y_train=y_train.values)

print(f"PCA降维结果已保存。原始特征维度: {X_train.shape[1]}, 降维后维度: {X_train_pca.shape[1]}")

In [ ]:
X_test = get_test_data(all_data_clean)
X_test_pca = pca_model.transform(X_test)
np.savez('./test_data_pca', X_test=X_test_pca)

In [ ]:
# 采用pca降维后的数据进行建模
train_data = np.load('./train_data_pca.npz')['X_train']
target_data = np.load('./train_data_pca.npz')['y_train']
# 切分数据
X_train, X_valid, y_train, y_valid = train_test_split(train_data, target_data, test_size=0.2)

##### 多元线性回归模型

In [ ]:
# 多元线性回归模型
clf = LinearRegression()
clf.fit(X_train, y_train)
score = mean_squared_error(y_valid, clf.predict(X_valid))
print("LinearRegression均方误差:", score)

In [ ]:
# 训练数据验证数据评估可视化
def plot_learning_curve(model, X_train, X_valid, y_train, y_valid):
    """
    绘制学习曲线，展示训练集和验证集的性能随训练样本数量的变化

    参数:
    model - 机器学习模型
    X_train - 训练集特征
    X_valid - 验证集特征
    y_train - 训练集目标变量
    y_valid - 验证集目标变量
    """

    # 初始化存储训练和验证分数的列表
    train_scores = []
    valid_scores = []
    for i in range(10, len(X_train) + 1, 10):
        model.fit(X_train[:i], y_train[:i])
        # 训练数据评估
        y_train_pred = model.predict(X_train[:i])
        train_scores.append(mean_squared_error(y_train[:i], y_train_pred))

        # 验证数据评估
        y_valid_pred = model.predict(X_valid)
        valid_scores.append(mean_squared_error(y_valid, y_valid_pred))

    # 可视化
    plt.plot([i for i in range(1, len(train_scores) + 1, 1)], train_scores, label='训练集')
    plt.plot([i for i in range(1, len(valid_scores) + 1, 1)], valid_scores, label='验证集')
    plt.legend()

In [ ]:
plot_learning_curve(LinearRegression(), X_train, X_valid, y_train, y_valid)
plt.savefig('./learning_curve.png', dpi=300, bbox_inches='tight')

##### K近邻回归模型

In [ ]:
# K近邻回归模型
for i in range(3, 20):
    clf = KNeighborsRegressor(n_neighbors=i)
    clf.fit(X_train, y_train)
    score = mean_squared_error(y_valid, clf.predict(X_valid))
    print(f"KNeighbors<UNK>{i}<UNK>: {score}")

##### 决策树回归模型

In [ ]:
# 决策树回归
clf = DecisionTreeRegressor()
clf.fit(X_train, y_train)
score = mean_squared_error(y_valid, clf.predict(X_valid))
print("DecisionTreeRegressor均方误差:", score)

##### 随机森林回归模型

In [ ]:
# 随机森林回归模型
clf = RandomForestRegressor(
    n_estimators=200,  # 设置森林中决策树的数量为200棵，增加模型稳定性和精度
    max_depth=10,  # 限制树的最大深度为10，防止过拟合
    min_samples_leaf=10,  # 每个叶节点最少包含10个样本，增强泛化能力
    min_samples_split=40,  # 分裂内部节点所需的最小样本数为40，控制树的生长
    max_features='sqrt',  # 每次分裂时考虑的特征数量为特征总数的平方根，增加模型多样性
    criterion='squared_error'  # 使用均方误差作为分裂质量的衡量标准
)

# 使用训练集拟合模型
clf.fit(X_train, y_train)

# 在验证集上进行预测并计算均方误差(MSE)评估模型性能
# 均方误差越小表示模型预测越准确
score = mean_squared_error(y_valid, clf.predict(X_valid))

# 输出模型在验证集上的均方误差
print("RandomForestRegressor均方误差:", score)

In [ ]:
# 随机森林回归模型评估可视化
plot_learning_curve(clf, X_train, X_valid, y_train, y_valid)

##### 向量机回归模型

In [ ]:
# 支持向量回归(SVR)模型训练与评估
# 创建第一个SVR模型实例 - 使用径向基函数(RBF)/高斯核
clf1 = SVR(
    kernel='rbf',  # 使用高斯径向基核函数，适合非线性数据建模
    C=1.0,  # 正则化参数，控制对误差的容忍度，值越小正则化越强
    gamma=0.01,  # 核函数系数，定义每个训练样本的影响范围，较小值意味着影响范围大
    tol=0.0001,  # 停止训练的误差阈值，提高精度但可能增加训练时间
    epsilon=0.3  # 不计入损失函数的误差带宽，调整模型对噪声的敏感度
)

# 使用训练数据拟合第一个SVR模型
clf1.fit(X_train, y_train)

# 在验证集上进行预测并计算均方误差，评估高斯核SVR性能
score = mean_squared_error(y_valid, clf1.predict(X_valid))

# 输出高斯核SVR模型的均方误差评估结果
print("支持向量机高斯核函数均方误差:", score)

# 创建第二个SVR模型实例 - 使用多项式核函数
clf2 = SVR(
    kernel='poly',  # 使用多项式核函数
    C=1.0,  # 正则化参数，控制模型复杂度与拟合质量的平衡
    degree=3,  # 多项式核函数的次数，影响模型的复杂度
    coef0=1.0,  # 核函数中的独立项系数，影响高阶项与低阶项的权重
    gamma='scale',  # 核函数系数，scale表示1/(n_features*X.var())
    epsilon=0.1,  # 不敏感区域宽度，小于epsilon的误差不会被考虑
    tol=0.0001  # 停止训练的误差阈值
)

# 使用训练数据拟合第二个SVR模型
clf2.fit(X_train, y_train)

# 在验证集上进行预测并计算均方误差，评估多项式核SVR性能
score = mean_squared_error(y_valid, clf2.predict(X_valid))

# 输出多项式核SVR模型的均方误差评估结果
print("支持向量机多项式核函数均方误差:", score)

In [ ]:
# 支持向量机SVR模型评估可视化
plot_learning_curve(clf1, X_train, X_valid, y_train, y_valid)

##### LightGBM梯度提升回归模型

In [ ]:
# LightGBM梯度提升回归模型
clf = lgb.LGBMRegressor(
    learning_rate=0.1,  # 学习率：控制每次迭代对模型的调整幅度，较高的值加快学习但可能导致过拟合
    n_estimators=500,  # 树的数量：设置为500棵，增加模型复杂度和学习能力
    max_depth=8,  # 树的最大深度：限制为5层以防止过拟合
    num_leaves=50,  # 叶子节点最大数量：控制树的复杂度，值越大模型越复杂
    min_child_samples=5,  # 叶子节点最小样本数：每个叶节点至少需要10个样本，提高稳定性
    subsample=0.8,  # 样本采样比例：每棵树随机使用80%的训练样本，减少过拟合风险
    colsample_bytree=0.8,  # 特征采样比例：每棵树随机使用80%的特征，增加模型多样性
    reg_alpha=0.1,  # L1正则化系数：控制模型稀疏性，有助于特征选择
    reg_lambda=0.05  # L2正则化系数：降低模型复杂度，防止过拟合
)

# 使用训练数据拟合LightGBM模型
clf.fit(X_train, y_train)

# 在验证集上评估模型性能，计算均方误差
score = mean_squared_error(y_valid, clf.predict(X_valid))

# 输出模型在验证集上的均方误差
print("LGBMRegressor均方误差:", score)

##### GBR梯度提升回归模型

In [ ]:
# 梯度提升回归(GBR)模型
clf = GradientBoostingRegressor(
    learning_rate=0.03,  # 学习率：控制每棵树对最终预测的贡献度，值较小可提高模型稳定性
    loss='huber',  # 损失函数：使用Huber损失，对异常值更具鲁棒性，平衡了均方误差和绝对误差的特点
    max_depth=14,  # 树的最大深度：限制为14层，增强模型复杂度和拟合能力
    max_features='sqrt',  # 特征选择方式：每次分裂时随机考虑特征总数的平方根，增加模型泛化能力
    min_samples_leaf=10,  # 叶节点最小样本数：每个叶节点至少需要10个样本，避免过拟合
    min_samples_split=40,  # 分裂节点所需最小样本数：内部节点分裂至少需要40个样本，控制树的生长
    n_estimators=300,  # 弱学习器(树)的数量：设置为300棵，通过集成多个模型提升预测精度
    subsample=0.8  # 样本采样比例：每棵树随机使用80%的训练样本，降低过拟合风险
)
clf.fit(X_train, y_train)
score = mean_squared_error(y_valid, clf.predict(X_valid))
print("GradientBoosting均方误差:", score)